# 데이터 로드 및 파싱

In [2]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-vkeif6at
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-vkeif6at
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 04aeebcbe26b109486a642e57dc58665c4818cf3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 72.7 MB/s et

In [3]:
import re
import json
import numpy as np
import pandas as pd

from pykospacing import Spacing

import torch
from torch.utils.data import  TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer

# 전처리

- 한 행에 한 문장 들어가도록

- clean_text: 텍스트 정제 함수
- InputExample: 텍스트, 라벨 연결해주는 클래스
- SequenceFeatures: 임베딩 시킨 피쳐들을 연결해주는 클래스
- convert_examples_to_seq_features: 문장을 임베딩한 피쳐로 변환하는 함수
- InferenceProcessor: test, label 열로 구성된 데이터프레임 넣어서 텍스트와 라벨을 연결해주는 클래스
- get_absa_loader: batch size에 따라 dataloader로 변환하는 함수

In [4]:
import ast

df_total = pd.DataFrame(columns=['text', 'labels'])

for i in range(1, 16):
  df_test = pd.read_csv('/content/drive/MyDrive/Customizing STTI/test/tag2label_'+str(i)+'.csv')
  df_total = pd.concat([df_total, df_test])

df_total.reset_index(drop=True, inplace=True)

for i in range(len(df_total)):
  raw_label = df_total.loc[i, 'labels']
  df_total.loc[i, 'labels'] = ast.literal_eval(raw_label)

df_total

,text,labels
0,유통기한도 넉넉하고 구성도 많아서 선물 하기 좋네요 만족합니다,"[B-POS, E-POS, B-POS, I-POS, I-POS, I-POS, E-P..."
1,구성도 알차고 촉촉하고 너무 좋아용,"[B-POS, E-POS, B-POS, I-POS, E-POS]"
2,대용량으로 넉넉하게 사용할 수 있고 무난하고 순한 편이네요 제품 구성은 좋으나 가격...,"[B-POS, I-POS, I-POS, I-POS, E-POS, B-POS, I-P..."
3,참존을 이 구성에 이 가격으로 사도 되나 싶은 생각이 드는 제품 양도 많아요,"[O, B-POS, I-POS, I-POS, I-POS, I-POS, I-POS, ..."
4,끈적임 없이 잘 흡수되어 매우 만족합니다,"[B-POS, E-POS, B-POS, I-POS, I-POS, E-POS]"
...,...,...
14995,얼굴에 열이 나면 트러블이 생겨서 지인 소개로 구입했습니다 ㅇㅇㅇ 마스트가 알로에 ...,"[O, O, O, O, O, O, O, O, O, O, B-NEG, I-NEG, I..."
14996,수분 부족 지성 피부입니다 사용해 봤는데 자극이 없어 좋습니다 팩하고 세안 했는데 ...,"[O, O, O, O, O, O, B-POS, I-POS, E-POS, O, O, ..."
14997,성분이 알로에라 촉촉함 하나는 기대했는데 촉촉하지가 않습니다 피부가 번들거리기만 해...,"[B-NEG, I-NEG, I-NEG, I-NEG, I-NEG, B-NEG, E-N..."
14998,잘 쓰고 있네요 트러블이 요즘 잦아서 쓰는 중인데 확실히 트러블이 가라앉아서 좋네요...,"[O, O, O, O, O, O, O, O, O, B-POS, I-POS, E-PO..."


In [ ]:
df_total[['text']].duplicated().sum()

0

In [5]:
for i in range(len(df_total)):
  text = df_total.loc[i, 'text']
  labels = df_total.loc[i, 'labels']

  assert len(text.split(' ')) == len(labels), i

# 임베딩

In [ ]:
label_list = processor.get_labels()
label_map = {label:i for i, label in enumerate(label_list)}
label_map

{'O': 0,
 'EQ': 1,
 'B-POS': 2,
 'I-POS': 3,
 'E-POS': 4,
 'S-POS': 5,
 'B-NEG': 6,
 'I-NEG': 7,
 'E-NEG': 8,
 'S-NEG': 9,
 'B-NEU': 10,
 'I-NEU': 11,
 'E-NEU': 12,
 'S-NEU': 13}

In [ ]:
label_map['O']

0

In [ ]:
a = ['B-POS', 'I-POS', 'E-POS', 'B-POS', 'I-POS', 'I-POS', 'E-POS', 'B-NEU', 'I-NEU', 'I-NEU', 'I-NEU', 'I-NEU', 'I-NEU', 'I-NEU', 'E-NEU']
label_ids = [label_map[label] for label in a]
label_ids

[2, 3, 4, 2, 3, 3, 4, 10, 11, 11, 11, 11, 11, 11, 12]

In [ ]:
len(label_ids)

15

In [5]:
def clean_text(raw_text):
  text = re.sub(r'[.]', ' ', raw_text)
  text = re.sub(r'[^ㄱ-ㅣ가-힣\s]', ' ', text)
  text = re.sub(' +', ' ', text)
  return text

class InputExample:
    def __init__(self, text_a, label=None):
        '''
        Inputs:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
                    sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            label: (Optional) string. The label of the example. This should be specified for train and dev examples.
        '''
        self.text_a = text_a
        self.label = label

In [6]:
class SeqInputFeatures:
    def __init__(self, input_ids, input_mask, segment_ids, label_ids, evaluate_label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.evaluate_label_ids = evaluate_label_ids

def convert_examples_to_seq_features(examples, label_list, tokenizer):
  label_map = {label:i for i, label in enumerate(label_list)}
  features = []
  max_seq_length = -1
  texts_tokenized = []

  cls_token='[CLS]'
  sep_token='[SEP]'
  cls_token_segment_id = 1
  sequence_a_segment_id = 0
  pad_token_segment_id = 0
  pad_token = 0

  for example in examples:
    tokens_total, labels_total = [], []
    evaluate_label_ids = []

    text = example.text_a
    words = example.text_a.split(' ')
    labels = example.label

    wid, tid = 0, 0
    for word, label in zip(words, example.label):   # 단어랑 태그를 하나씩 묶음
      subwords = tokenizer.tokenize(word)           # 단어 토크나이징 -> 서브단어 생길 수 있음
      tokens_total.extend(subwords)                 # 최종 토큰에 붙여 넣음
      if label != 'O':                              # 라벨이 O이 아니면
        labels_total.extend([label] + (['EQ'] * (len(subwords) - 1)))   # 서브워드 생긴만큼 EQ 태그를 뒤에 삽입
      else:                                         # 라벨이 O 이면
        labels_total.extend(['O'] * len(subwords))  # 서브워드 길이만큼 O 삽입
      evaluate_label_ids.append(tid)                # evaluate_label_ids에 tid 삽입
      wid += 1                                      # wid 1 증가
      tid += len(subwords)                          # 서브워드 길이만큼 tid 증가
    assert tid == len(tokens_total)                 # 최종 tid가 토큰수와 같은지

    evaluate_label_ids = np.array(evaluate_label_ids, dtype=np.int32) # 어레이로 바꿈? (어레이: 연속적으로 저장)
    texts_tokenized.append((tokens_total, labels_total, evaluate_label_ids)) # 종합해서 저장

    if len(tokens_total) > max_seq_length:          # 토큰 길이가 최대 문장 길이보다 길면
      max_seq_length = len(tokens_total)            # 최대 문장 길이 갱신

  assert len(examples) == len(texts_tokenized)
  max_seq_length += 2                               # cls, sep 토큰 2개 추가
  for idx, (tokens_a, labels_a, evaluate_label_ids) in enumerate(texts_tokenized):
    tokens = ['[CLS]'] + tokens_a + ['[SEP]']
    labels = ['O'] + labels_a  + ['O']
    segment_ids = [cls_token_segment_id] + [sequence_a_segment_id] * (len(tokens)-1)
    evaluate_label_ids += 1

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    padding_length = max_seq_length - len(input_ids)
    label_ids = [label_map[label] for label in labels]

    input_ids = input_ids + ([pad_token] * padding_length)
    input_mask = input_mask + ([0] * padding_length)
    segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    label_ids = label_ids + ([0] * padding_length)

    assert len(input_ids) == max_seq_length,    text
    assert len(input_mask) == max_seq_length,   text
    assert len(segment_ids) == max_seq_length,  text
    assert len(label_ids) == max_seq_length,    labels_a

    features.append(SeqInputFeatures(input_ids=input_ids,
                                     input_mask=input_mask,
                                     segment_ids=segment_ids,
                                     label_ids=label_ids,
                                     evaluate_label_ids=evaluate_label_ids))
  #print('maximal sequence length is %d' % (max_seq_length))

  return features

In [19]:
processor = DataProcessor()
label_list = processor.get_labels()
num_labels = len(label_list)

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

error_list = []

for i in range(len(df_total)):
  try:
    examples_t = processor.get_examples(df_total.loc[i:i, :])
    features_t = convert_examples_to_seq_features(examples=examples_t, label_list=label_list, tokenizer=tokenizer)
  except AssertionError as e:
    error_list.append(i)

In [20]:
error_list

[599,
 1846,
 2968,
 3845,
 4057,
 4296,
 4414,
 5203,
 5443,
 7322,
 11962,
 12539,
 12565,
 13137,
 14680]

In [21]:
len(error_list)

15

In [22]:
df_total.drop(error_list, inplace=True)

In [23]:
len(df_total)

14985

In [43]:
torch.save(features_t, '/content/test_data.pt')

In [8]:
'''
tensor([[    2,  4743,  2015,  ...,     0,     0,     0],
        [    2,  3896,  2119,  ...,     0,     0,     0],
        [    2, 18059,  6233,  ...,     0,     0,     0],
        ...,
        [    2,  3977,  2170,  ...,     0,     0,     0],
        [    2,  5411, 31221,  ...,     0,     0,     0],
        [    2,  1443,  4514,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]])'''

class DataProcessor:
    def get_examples(self, dataframe):
        return self._create_examples(dataframe)

    def get_labels(self):
        return ['O', 'EQ', 'B-POS', 'I-POS', 'E-POS', 'S-POS',
                 'B-NEG', 'I-NEG', 'E-NEG', 'S-NEG',
                 'B-NEU', 'I-NEU', 'E-NEU', 'S-NEU']

    def _create_examples(self, dataframe):
        examples = []
        dataframe.reset_index(drop=True, inplace=True)

        for sample_id, sentence in enumerate(dataframe.loc[:, 'text'].values):
            label = dataframe.loc[sample_id, 'labels']
            examples.append(InputExample(text_a = sentence, label=label))

        return examples

def get_absa_loader(dataframe, tokenizer, mode, batch_size):
  processor = DataProcessor()
  examples = processor.get_examples(dataframe)
  label_list = processor.get_labels()
  features = convert_examples_to_seq_features(examples=examples, label_list=label_list, tokenizer=tokenizer)
  torch.save(features, '/content/'+str(mode)+'.pt')

  all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
  all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
  all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

  all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)
  all_evaluate_label_ids = [f.evaluate_label_ids for f in features]
  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

  if mode=='train':
    sampler = RandomSampler(dataset)
  elif mode=='valid' or mode=='test':
    sampler = SequentialSampler(dataset)

  dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)

  return (dataloader, all_evaluate_label_ids)

# BERT layer

In [9]:
from transformers import PreTrainedModel, BertModel, BertConfig
from torch.utils.data import Dataset

import torch
from torch import nn
from transformers import BertModel

- ABSAProcessor: 논문 태그로 바꾸고 example 생성
- InferenceProcessor: 이미 논문 태그로 바뀌어 있는 dataframe 사용해서 example 생성

- 모델을 학습시키기 위한 요소들을 명시한 json 파일로 되어 있음
- batch size, learning rate, weigth_decay 등 train에 필요한 요소들부터 tokenizer에 특수 토큰 (special token eg[mask] ) 들을 미리 설정하는 등 설정에 관한 전반적인 것들 명시
- PretrainedModel을 save_pretrained 메소드를 이용하면 모델의 체크포인트와 함께 저장되도록 되어있음.
- hugging face의 pretrained model을 그대로 사용하게 되면 자동으로 config파일이 로드되어 명시할 필요가 없지만, 설정을 변경하고 싶거나 나만의 모델을 학습시킬 때에는 config파일을 직접 불러와야 함.
- config 또한 model, tokenizer처럼 Model ID만 있으면, Config 클래스를 명확히 지정하거나 혹은 AutoConfig를 이용하는 방식으로 불러올 수 있음

- 기본적으로 모델들은 PretrainedModel 클래스를 상속받고 있음
- PretrainedModel 클래스는 학습된 모델을 불러오고, 다운로드하고, 저장하는 등 모델 전반에 걸쳐 적용되는 메소드를 가지고 있음
- 상속 구조를 가지고 있기 때문에, 실제로 사용할 모델이 BERT 이건 GPT 이건 상관없이 모델을 불러오고 다운로드/저장하는 등의 작업에 활용하는 메소드는 부모 클래스의 것을 동일하게 활용

https://velog.io/@olxtar/PyTorch-class%EB%A1%9C-%EC%83%9D%EC%84%B1%EB%90%9C-model%EC%9D%98-layer-weights-%EC%A0%91%EA%B7%BC%EB%B2%95

https://tutorials.pytorch.kr/beginner/former_torchies/tensor_tutorial_old.html

In [10]:
class TaggerConfig:
    def __init__(self):
        self.hidden_dropout_prob = 0.1
        self.hidden_size = 768

class BertLayerNorm(nn.Module): # hiddem layer 전체의 평균과 분산으로 normalization함
    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size)) # torch.nn.parameter.Parameter
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps # 분모가 0이 되는 경우를 방지하기 위한 아주 작은 상수

    def forward(self, x): # input을 넣고 어떤 과정을 거쳐 output이 나올지를 정의해준다는 느낌
        u = x.mean(-1, keepdim=True) # 입력값 x(텐서)의 마지막 차원 (-1)을 따라 평균을 구하고 차원을 유지하는 함수. keepdim=True인 경우 출력 텐서는 크기가 1인 dim 차원을 제외하고 input과 동일한 크기
        s = (x - u).pow(2).mean(-1, keepdim=True) # 왜 -1 쓰는지? -> 열벡터로 출력하기 위해
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias # 일종의 예측값(y_hat)으로 볼 수 있다.

class BertPreTrainedModel(PreTrainedModel):
    config_class = BertConfig
    base_model_prefix = "bert"

    def __init__(self, *inputs, **kwargs):
        super(BertPreTrainedModel, self).__init__(*inputs, **kwargs)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range) # 가중치 초기화
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

class BertABSATagger(BertPreTrainedModel):
    def __init__(self, bert_config): # config 불러옴
        super(BertABSATagger, self).__init__(bert_config)
        self.num_labels = bert_config.num_labels
        self.tagger_config = TaggerConfig()
        self.bert = BertModel(bert_config) # 해당 config로 모델을 생성
        self.bert_dropout = nn.Dropout(bert_config.hidden_dropout_prob) # 드랍아웃 설정
        if bert_config.fix_tfm:
            for p in self.bert.parameters():
                p.required_grad = False  # Frizen 미분 안되도록?

        self.tagger_dropout = nn.Dropout(self.tagger_config.hidden_dropout_prob) # tagger 드랍 아웃이 왜 따로 있지?
        self.tagger = SAN(model=bert_config.hidden_size, nhead=12, dropout=0.1)
        penultimate_hidden_size = self.tagger_config.hidden_size
        self.classifier = nn.Linear(penultimate_hidden_size, self.num_labels) # 마지막 레이어

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, position_ids=None, head_mask=None):
        outputs = self.bert(
            input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            head_mask=head_mask
        ) # 정의한 bert 모델에 입력값 넣음 -> 넣으면 뭐가 되는데?
        # Transformer의 인코더를 이용하여 구축된 BERT를 사용

        # 기존 연구에서 제안된 E2E-ABSA 모델과 같이 BERT를 사용하고 마지막에 single sequence tagger를 통과해 속성을 예측
        tagger_input = outputs[0] # pooler
        tagger_input = self.bert_dropout(tagger_input)
        tagger_input = tagger_input.transpose(0, 1) # 각 성분에 대해서 classification을 하기 위함.
        classifier_input = self.tagger(tagger_input)
        classifier_input = classifier_input.transpose(0, 1)
        classifier_input = self.tagger_dropout(classifier_input)
        logits = self.classifier(classifier_input) # 소프트맥스 통과는 어디?

        outputs = (logits, ) + outputs[2:]

        if labels is not None:
            criterion = nn.CrossEntropyLoss()

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = criterion(active_logits, active_labels)

            else:
                loss = criterion(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss, ) + outputs

        return outputs

class SAN(nn.Module):
    def __init__(self, model, nhead, dropout=0.1):
        super(SAN, self).__init__()
        self.model = model
        self.self_attn = nn.MultiheadAttention(model, nhead, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)
        self.norm = nn.LayerNorm(model)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask) # (key, query, value)
        src = src + self.dropout(src2)
        src = self.norm(src)
        return src

# train

os 모듈은 Operating System의 약자로서 운영체제에서 제공되는 여러 기능을 파이썬에서 수행할 수 있게 해줍니다.

예를 들어, 파이썬을 이용해 파일을 복사하거나 디렉터리를 생성하고 특정 디렉터리 내의 파일 목록을 구하고자 할 때 os 모듈을 사용하면 됩니다.

In [11]:
import os
import random
import easydict
from tqdm import tqdm
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup

In [12]:
SMALL_POSITIVE_CONST = 1e-4

args = easydict.EasyDict({"absa_type": 'san',
                          "model_type": 'bert',
                          "fix_tfm": 0,
                          "max_seq_length": 512,
                          "num_epochs": 100,
                          "batch_size": 128,
                          "save_steps": 100,
                          "seed": 42,
                          "epsilon": 1e-8,
                          "warmup_steps": 0,
                          "model_name_or_path": 'klue/bert-base',
                          "max_grad_norm": 1.0,
                          "learning_rate": 1e-5,
                          "device": 'cuda'})

BASE_DIR = os.getcwd()
#SEM_DIR = os.path.join(BASE_DIR, 'SemEval')
#YELP_DIR = os.path.join(BASE_DIR, 'Yelp')
PARAM_DIR = os.path.join(BASE_DIR, 'bert-san')
RESULT_DIR = os.path.join(BASE_DIR, 'results')

In [13]:
def match_ts(gold_ts_sequence, pred_ts_sequence):
    '''
    Inputs:
        gold_ts_sequence: gold standard targeted sentiment sequence (ground truth)
        pred_ts_sequence: predicted targeted sentiment sequence
    '''

    tag2tagid = {'POS': 0, 'NEG': 1, 'NEU': 2}
    hit_count, gold_count, pred_count = np.zeros(3), np.zeros(3), np.zeros(3)
    for t in gold_ts_sequence:
        ts_tag = t[2]
        tid = tag2tagid[ts_tag]
        gold_count[tid] += 1

    for t in pred_ts_sequence:
        ts_tag = t[2]
        tid = tag2tagid[ts_tag]
        if t in gold_ts_sequence:
            hit_count[tid] += 1
        pred_count[tid] += 1
    return hit_count, gold_count, pred_count

def compute_metrics_absa(preds, labels, all_evaluate_label_ids):
    absa_label_vocab = {'O':0, 'EQ':1, 'B-POS':2, 'I-POS':3, 'E-POS':4, 'S-POS':5,
                        'B-NEG':6, 'I-NEG':7, 'E-NEG':8, 'S-NEG':9, 'B-NEU':10,
                        'I-NEU':11, 'E-NEU':12, 'S-NEU':13}
    absa_id2tag = {}
    for k in absa_label_vocab:
        v = absa_label_vocab[k]
        absa_id2tag[v] = k

    n_tp_ts, n_gold_ts, n_pred_ts = np.zeros(3), np.zeros(3), np.zeros(3)
    ts_precision, ts_recall, ts_f1 = np.zeros(3), np.zeros(3), np.zeros(3)
    n_samples = len(all_evaluate_label_ids)

    class_count = np.zeros(3)

    for i in range(n_samples):
        evaluate_label_ids = all_evaluate_label_ids[i]
        pred_labels = preds[i][evaluate_label_ids]
        gold_labels = labels[i][evaluate_label_ids]
        assert len(pred_labels) == len(gold_labels)

        pred_tags = [absa_id2tag[label] for label in pred_labels]
        gold_tags = [absa_id2tag[label] for label in gold_labels]

        g_ts_sequence, p_ts_sequence = tag2ts(ts_tag_sequence=gold_tags), tag2ts(ts_tag_sequence=pred_tags)

        hit_ts_count, gold_ts_count, pred_ts_count = match_ts(gold_ts_sequence=g_ts_sequence,
                                                              pred_ts_sequence=p_ts_sequence)

        n_tp_ts += hit_ts_count
        n_gold_ts += gold_ts_count
        n_pred_ts += pred_ts_count

        for (_, _, s) in g_ts_sequence:
            if s == 'POS':
                class_count[0] += 1
            if s == 'NEG':
                class_count[1] += 1
            if s == 'NEU':
                class_count[2] += 1

    for i in range(3):
        n_ts = n_tp_ts[i]
        n_g_ts = n_gold_ts[i]
        n_p_ts = n_pred_ts[i]
        ts_precision[i] = float(n_ts) / float(n_p_ts +SMALL_POSITIVE_CONST)
        ts_recall[i] = float(n_ts) / float(n_g_ts + SMALL_POSITIVE_CONST)
        ts_f1[i] = 2 * ts_precision[i] * ts_recall[i] / (ts_precision[i] + ts_recall[i] + SMALL_POSITIVE_CONST)

    macro_f1 = ts_f1.mean()

    n_tp_total = sum(n_tp_ts)
    n_g_total = sum(n_gold_ts)
    print('class_count:', class_count)

    n_p_total = sum(n_pred_ts)
    micro_p = float(n_tp_total) / (n_p_total + SMALL_POSITIVE_CONST)
    micro_r = float(n_tp_total) / (n_g_total + SMALL_POSITIVE_CONST)
    micro_f1 = 2 * micro_p * micro_r / (micro_p + micro_r + SMALL_POSITIVE_CONST)
    scores = {'macro-f1':macro_f1, 'precision':micro_p, 'recall':micro_r, 'micro-f1':micro_f1}

    return scores

def set_seed(args):
  random.seed(args.seed)
  np.random.seed(args.seed)
  torch.manual_seed(args.seed)
  torch.cuda.manual_seed_all(args.seed)

In [14]:
def tag2ts(ts_tag_sequence): # ts_tag_sequence: [O, O, S-NEG, O, O, O, O, O, O]
    n_tags = len(ts_tag_sequence) # 9
    ts_sequence, sentiments = [], []
    beg, end = -1, -1

    for i in range(n_tags):
        ts_tag = ts_tag_sequence[i]
        eles = ts_tag.split('-') # ['O'] 혹은 ['S', 'NEG']

        if len(eles) == 2: # 태그가 O이 아니면
            pos, sentiment = eles
            # pos: S
            # sentiment: NEG
        else: # 태그가 O이면
            pos, sentiment = 'O', 'O'

        if sentiment != 'O': # 태그가 O이 아니면, sentiment: NEG
            sentiments.append(sentiment) # sentiment = ['NEG']

        if pos == 'S': # Single tag이면
            ts_sequence.append((i, i, sentiment)) # 튜플 추가, [(2, 2, 'NEG')]
            sentiments = [] # 초기화

        elif pos == 'B': # Beginning tag이면
            beg = i
            if len(sentiments) > 1: # 앞의 태그가 O이 여러개였다면 ['O', ..., 'O'] 이렇게 붙었을 것
                sentiments = [sentiments[-1]] # 마지막 한 개만 가져옴

        elif pos == 'E': # Ending tag이면
            end = i

            if end > beg > -1 and len(set(sentiments)) == 1:
                ts_sequence.append((beg, end, sentiment)) # 위치표시, [(3, 5, 'NEG')]
                sentiments = []
                beg, end = -1, -1

    return ts_sequence # [(2, 2, 'NEG'), (3, 5, 'NEG')]

In [15]:
# TaggerConfig
# BertLayerNorm
# BertPreTrainedModel
# BertABSATagger

def absa_train(args, model, train_loader, eval_loader, optimizer):

    t_total = len(train_loader) * args.num_epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
    best_loss = float(np.inf)
    global_step = 0
    train_loss_list, eval_loss_list = [], []
    results_list = []
    set_seed(args)
    print('***** Running Training *****')
    for epoch in range(1, args.num_epochs + 1):
        train_loss, eval_loss = 0.0, 0.0
        train_iterator = tqdm(train_loader, desc='training...', disable=False)
        model.train()

        for batch in train_iterator:

            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':          batch[0],
                      'attention_mask':     batch[1],
                      'token_type_ids':     batch[2],
                      'labels':             batch[3]}

            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs[0]
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)

            train_loss += loss.item() / len(batch)
            optimizer.step()
            scheduler.step()
            global_step += 1


            # model check-point
            if global_step % args.save_steps == 0:
                output_dir = os.path.join(args.output_dir, f'checkpoint-{global_step}')
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                model_to_save = model.module if hasattr(model, 'module') else model
                model_to_save.save_pretrained(output_dir)
                torch.save(args, os.path.join(output_dir, 'training_args.bin'))


            train_loss /= len(train_iterator)
            train_loss_list.append(train_loss)



        model.eval()
        eval_iterator = tqdm(eval_loader[0], desc='training...')
        preds, out_label_ids = None, None
        results = {}
        for batch in eval_iterator:
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':          batch[0],
                        'attention_mask':     batch[1],
                        'token_type_ids':     batch[2],
                        'labels':             batch[3]}

                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]
                eval_loss += tmp_eval_loss.mean().item()

            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()

            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss /= len(eval_iterator)

        eval_loss_list.append(eval_loss)
        preds = np.argmax(preds, axis=-1)
        result = compute_metrics_absa(preds, out_label_ids, eval_loader[1])
        results.update(result)
        results_list.append(results)

        output_file = os.path.join(args.output_dir, '%s_results.txt' % 'valid')
        if not os.path.exists(args.output_dir):
            os.makedirs(args.output_dir)

        with open(output_file, 'w') as writer:
            for key in sorted(result.keys()):
                if 'eval_loss' in key:
                    print(f'{key} = {str(result[key])}')
                writer.write('%s = %s\n' % (key, str(result[key])))

        if best_loss > eval_loss :
            torch.save(model.state_dict(), os.path.join(args.output_dir, f'best-parameters.pt'))
            best_loss = eval_loss
            best_epoch = epoch

        print(f'epochs: [{epoch}/{args.num_epochs}], best epoch is {best_epoch}')
        print(f'train loss: {train_loss:.5f},\t valid loss: {eval_loss:.5f}')
        print(f'macro-f1: {result["macro-f1"]:.5f},\tmicro-f1: {result["micro-f1"]:.5f}')
        print(f"recall: {result['recall']:.5f},\tprecision: {result['precision']:.5f}")
    print('***** Finished Training *****')

    return {
        'results': results_list,
        'train_loss': train_loss_list,
        'valid_loss': eval_loss_list
    }

In [16]:
import pickle
from torch import optim

In [24]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_total, test_size=0.2, random_state=11)

df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [25]:
output_dir = f'{args.model_type}-{args.absa_type}'
args.output_dir = output_dir

processor = DataProcessor()
label_list = processor.get_labels()
num_labels = len(label_list)

config_class, model_class, tokenizer_class = BertConfig, BertABSATagger, BertTokenizer
config = config_class.from_pretrained(args.model_name_or_path, num_labels=num_labels)
tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)

config.absa_type = args.absa_type
config.fix_tfm = args.fix_tfm

model = model_class.from_pretrained(args.model_name_or_path, config=config)

model = model.to(args.device)
batch_size = 64
train_dataloader, _ = get_absa_loader(df_train, tokenizer, mode='train', batch_size=batch_size)
valid_dataloader = get_absa_loader(df_test, tokenizer, mode='valid', batch_size=batch_size)
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)

results = absa_train(args, model, train_dataloader, valid_dataloader, optimizer)

save_dir = os.path.join(RESULT_DIR, '/content/drive/MyDrive/Customizing STTI/test/valid_results.pkl')
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

with open(save_dir, 'wb') as f:
    pickle.dump(results, f)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertABSATagger were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['tagger.self_attn.out_proj.bias', 'tagger.self_attn.out_proj.weight', 'classifier.weight', 'tagger.norm.bias', 'classifier.bias', 'tagger.norm.weight', 'tagger.self_attn.in_proj_bias', 'tagger.self_attn.in_proj_weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


***** Running Training *****


training...:   0%|          | 0/188 [00:01<?, ?it/s]


OutOfMemoryError: ignored

In [155]:
df_total.loc[1613, 'text']

'촉촉하고 윤기가 있어 보여 좋아요 지인들에게 선물하고 싶어요'

In [147]:
df_train.loc[500, 'text']

'사춘기 딸이 후기 보고 구입했는데 여드름에 효과 있어요 아이가 세안할 때 한번 누르기만 하면 거품이 풍부하게 나와서 시간도 단축되고 사용하기 편리해서 다른 제품보다 이 제품에 손이 가네요 헹굴 때도 미끌거리 지 않고 잘 헹궈져서 좋아요 단점은 소독할 때 나는 알콜향이 나 고 지성인 부분 외에 다른 피부를 건조하게 하네요 가격도 다른 여드름 상품 치고는 비싸지 않은 편이고  대용량이라 좋아요 붉은 여드름에도 진정 효과가 있지만 블랙헤드에는 효과가 없어요 눈가는 피해서 폼을 사용해야 해요'

In [148]:
len(df_train.loc[500, 'text'].split(' '))

73

In [150]:
over = []
for i in range(len(df_total)):
  if len(df_total.loc[i, 'text'].split(' ')) > 70:
    over.append(i)
print(len(over))

130


In [151]:
over

[130,
 594,
 944,
 1185,
 1385,
 1405,
 1504,
 1719,
 1882,
 1908,
 1911,
 1920,
 2212,
 2584,
 2641,
 2682,
 2701,
 2915,
 2924,
 3040,
 3171,
 3289,
 3354,
 3641,
 3681,
 3791,
 3806,
 3845,
 3890,
 3892,
 3902,
 3964,
 3969,
 4040,
 4097,
 4099,
 4140,
 4234,
 4271,
 4275,
 4294,
 4543,
 4564,
 4589,
 4620,
 4640,
 4670,
 4707,
 4719,
 4775,
 4900,
 4916,
 5101,
 5183,
 5195,
 5196,
 5203,
 5213,
 5324,
 5329,
 5346,
 5369,
 5432,
 5443,
 5495,
 5545,
 5945,
 6211,
 6525,
 6545,
 6552,
 6591,
 6643,
 6887,
 7169,
 7226,
 7277,
 7278,
 7330,
 7343,
 7363,
 7371,
 7404,
 7523,
 7531,
 7583,
 7649,
 7747,
 7751,
 7768,
 7832,
 7964,
 7974,
 8292,
 8349,
 8371,
 8411,
 8471,
 8593,
 8619,
 8750,
 8859,
 11681,
 11720,
 11764,
 12176,
 12493,
 12539,
 12561,
 12565,
 12567,
 12601,
 12620,
 12652,
 12654,
 12656,
 12678,
 12712,
 12729,
 12748,
 13115,
 13144,
 13936,
 14035,
 14118,
 14227,
 14264,
 14506,
 14509,
 14989]

In [152]:
df_total.drop(over, inplace=True)
df_total.reset_index(inplace=True, drop=True)
df_total

,text,labels
0,유통기한도 넉넉하고 구성도 많아서 선물 하기 좋네요 만족합니다,"[B-POS, E-POS, B-POS, I-POS, I-POS, I-POS, E-P..."
1,구성도 알차고 촉촉하고 너무 좋아용,"[B-POS, E-POS, B-POS, I-POS, E-POS]"
2,대용량으로 넉넉하게 사용할 수 있고 무난하고 순한 편이네요 제품 구성은 좋으나 가격...,"[B-POS, I-POS, I-POS, I-POS, E-POS, B-POS, I-P..."
3,참존을 이 구성에 이 가격으로 사도 되나 싶은 생각이 드는 제품 양도 많아요,"[O, B-POS, I-POS, I-POS, I-POS, I-POS, I-POS, ..."
4,끈적임 없이 잘 흡수되어 매우 만족합니다,"[B-POS, E-POS, B-POS, I-POS, I-POS, E-POS]"
...,...,...
14865,얼굴에 열이 나면 트러블이 생겨서 지인 소개로 구입했습니다 ㅇㅇㅇ 마스트가 알로에 ...,"[O, O, O, O, O, O, O, O, O, O, B-NEG, I-NEG, I..."
14866,수분 부족 지성 피부입니다 사용해 봤는데 자극이 없어 좋습니다 팩하고 세안 했는데 ...,"[O, O, O, O, O, O, B-POS, I-POS, E-POS, O, O, ..."
14867,성분이 알로에라 촉촉함 하나는 기대했는데 촉촉하지가 않습니다 피부가 번들거리기만 해...,"[B-NEG, I-NEG, I-NEG, I-NEG, I-NEG, B-NEG, E-N..."
14868,잘 쓰고 있네요 트러블이 요즘 잦아서 쓰는 중인데 확실히 트러블이 가라앉아서 좋네요...,"[O, O, O, O, O, O, O, O, O, B-POS, I-POS, E-PO..."


In [ ]:
df_train[df_train['text']=='보습이 잘 되네요 화장하니 얼굴에 광이나서 좋습니다 피부 결 모공도 개선 효과 있기를 기대해 봅니다']

In [ ]:
df_train.loc[11999, 'labels']

In [ ]:
len(df_train.loc[11999, 'text'].split(' '))

15

In [ ]:
len(df_train.loc[11999, 'labels'])

15

In [ ]:
df_train[df_train['text']=='보습이 잘 되네요 화장하니 얼굴에 광이나서 좋습니다 피부 결 모공도 개선 효과 있기를 기대해 봅니다']

,text,labels


# 참고

In [ ]:
def ot2bieos_ts(ts_tag_sequence): # ts_tag_sequence: # ['O', 'O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O']
    n_tags = len(ts_tag_sequence) # n_tags: 9
    new_ts_sequence = []
    prev_pos = '$$$'

    for i in range(n_tags):
        cur_ts_tag = ts_tag_sequence[i]
        if cur_ts_tag == 'O' or cur_ts_tag == 'EQ':
            new_ts_sequence.append('O')
            cur_pos = 'O'

        else: # 'T-NEG'
            cur_pos, cur_sentiment = cur_ts_tag = cur_ts_tag.split('-')
            # cur_ts_tag: ['T', 'NEG']
            # cur_pos: T
            # cur_sentiment: NEG

            if cur_pos != prev_pos: # 현재 위치

                if i == n_tags -1: # 마지막 태그이면
                    new_ts_sequence.append('S-%s' % cur_sentiment) # Single tag

                else: # 마지막 태그가 아니면
                    next_ts_tag = ts_tag_sequence[i + 1] # 다음 태그 추출
                    if next_ts_tag == 'O': # 다음 태그가 O이면 Single tag
                        new_ts_sequence.append('S-%s' % cur_sentiment)

                    else: # 다음 태그가 O이 아니면 Beginning tag
                        new_ts_sequence.append('B-%s' % cur_sentiment)

            else: # cur_pos == prev_pos
                if i == n_tags -1: # 마지막 태그이면 Ending tag
                    new_ts_sequence.append('E-%s' % cur_sentiment)

                else: # 마지막 태그가 아니면
                    next_ts_tag = ts_tag_sequence[i + 1] # 다음 태그 추출
                    if next_ts_tag == 'O': # 다음 태그가 O이면 Ending tag
                        new_ts_sequence.append('E-%s' % cur_sentiment)
                    else: # 다음 태그가 O이 아니면 Inside tag
                        new_ts_sequence.append('I-%s' % cur_sentiment)
        prev_pos = cur_pos
    return new_ts_sequence

def tag2ts(ts_tag_sequence): # ts_tag_sequence: [O, O, S-NEG, O, O, O, O, O, O]
    n_tags = len(ts_tag_sequence) # 9
    ts_sequence, sentiments = [], []
    beg, end = -1, -1

    for i in range(n_tags):
        ts_tag = ts_tag_sequence[i]
        eles = ts_tag.split('-') # ['O'] 혹은 ['S', 'NEG']

        if len(eles) == 2: # 태그가 O이 아니면
            pos, sentiment = eles
            # pos: S
            # sentiment: NEG
        else: # 태그가 O이면
            pos, sentiment = 'O', 'O'

        if sentiment != 'O': # 태그가 O이 아니면, sentiment: NEG
            sentiments.append(sentiment) # sentiment = ['NEG']

        if pos == 'S': # Single tag이면
            ts_sequence.append((i, i, sentiment)) # 튜플 추가, [(2, 2, 'NEG')]
            sentiments = [] # 초기화

        elif pos == 'B': # Beginning tag이면
            beg = i
            if len(sentiments) > 1: # 앞의 태그가 O이 여러개였다면 ['O', ..., 'O'] 이렇게 붙었을 것
                sentiments = [sentiments[-1]] # 마지막 한 개만 가져옴

        elif pos == 'E': # Ending tag이면
            end = i

            if end > begin > -1 and len(set(sentiments)) == 1:
                ts_sequence.append((begin, end, sentiment)) # 위치표시, [(3, 5, 'NEG')]
                sentiments = []
                begin, end = -1, -1

    return ts_sequence # [(2, 2, 'NEG'), (3, 5, 'NEG')]

class InputExample:
    def __init__(self, guid, text_a, text_b=None, label=None):
        '''
        Inputs:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
                    sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            label: (Optional) string. The label of the example. This should be specified for train and dev examples.
        '''
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

class ABSAProcessor:
    def get_examples(self, data_dir, set_type='train'):
        return self._create_examples(data_dir=data_dir, set_type=set_type)

    def get_labels(self):
        return ['O', 'EQ', 'B-POS', 'I-POS', 'E-POS', 'S-POS',
                 'B-NEG', 'I-NEG', 'E-NEG', 'S-NEG',
                 'B-NEU', 'I-NEU', 'E-NEU', 'S-NEU']

    def _create_examples(self, data_dir, set_type):
        examples = []
        file = os.path.join(data_dir, '%s.txt' % set_type) # train.txt, test.txt 파일 불러옴
        class_count = np.zeros(3) # 긍정, 부정, 중립 개수 세기 위해
        with open(file, 'r', encoding='UTF-8') as fp:
            sample_id = 0
            for line in fp:
                '''
                sent_string: But the staff was so horrible to us.
                tag_string: But=O the=O staff=T-NEG was=O so=O horrible=O to=O us=O .=O

                words: [But, the, staff, was, so, horrible, to, us, .]
                tags: [O, O, S-NEG, O, O, O, O, O, O]
                '''
                sent_string, tag_string = line.strip().split('####') # strip() 양쪽 끝에 공백 제거
                # sent_stirng: But the staff was so horrible to us.
                # tag_string: But=O the=O staff=T-NEG was=O so=O horrible=O to=O us=O .=O

                words, tags =[], []
                for tag_item in tag_string.split(' '): # tag_item: ['But=O', 'the=O', ...]
                    eles = tag_item.split('=') # ['But', 'O']
                    if len(eles) == 1:
                        raise Exception('Invalid samples %s...' % tag_string) # 오류

                    elif len(eles) == 2:
                        word, tag = eles
                        # word: But
                        # tag: O

                    else:
                        word = ''.join((len(eles) - 2) * ['=']) # else에서 2 뺀 만큼의 =를 word로 지정
                        tag = eles[-1] # else에서 마지막 원소를 tag로 지정

                    words.append(word) # [But, the, staff, was, so, horrible, to, us, .]
                    tags.append(tag) # ['O', 'O', 'T-NEG', 'O', 'O', 'O', 'O', 'O', 'O'], Semuel 태그

                tags = ot2bieos_ts(tags) # ['O', 'O', 'S-NEG', 'O', 'O', 'O', 'O', 'O', 'O'], 논문 태그로 바꿈

                guid = '%s-%s' % (set_type, sample_id) # train-0
                text_a = ' '.join(words) # But the staff was so horrible to us .
                gold_ts = tag2ts(ts_tag_sequence=tags) # [(2, 2, 'NEG'), (3, 5, 'NEG')]
                for (_, _, s) in gold_ts:
                    if s == 'POS':
                        class_count[0] += 1

                    if s == 'NEG':
                        class_count[1] += 1

                    if s == 'NEU':
                        class_count[2] += 1

                examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=tags))
                sample_id += 1


            print('%s class count: %s' % (set_type, class_count))
            print(f'review length: {sample_id:,}\t total class count: {int(sum(class_count)):,}')
            return examples

#===============================================================================

class SeqInputFeatures:
    def __init__(self, input_ids, input_mask, segment_ids, label_ids, evaluate_label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.evaluate_label_ids = evaluate_label_ids

def convert_examples_to_seq_features(examples, label_list, tokenizer,
                                     cls_token='[CLS]', sep_token='[SEP]', pad_token=0, sequence_a_segment_id=0,
                                     cls_token_segment_id=1, pad_token_segment_id=0, mask_padding_with_zero=True):

    label_map = {label:i for i, label in enumerate(label_list)} # {'O':0, 'EQ':1, 'B-POS':2, 'I-POS':3, ...
    features = [] # 초기화
    max_seq_length = -1 # 초기화
    examples_tokenized = [] # 초기화
    for example in examples:
        tokens_a, labels_a = [], [] # 초기화
        evaluate_label_ids = [] # 초기화
        words = example.text_a.split(' ') # [But, the, staff, was, so, horrible, to, us, .]
        wid, tid = 0, 0
        for word, label in zip(words, example.label): # But O
            subwords = tokenizer.tokenize(word) # ! 단어별 토근화
            tokens_a.extend(subwords) # 토큰화한 문장
            if label != 'O':
                labels_a.extend([label] + ['EQ'] * (len(subwords) - 1)) # [B-POS, EQ, EQ]
            else: # 라벨이 O이면
                labels_a.extend(['O'] * len(subwords)) # subwords 개수만큼 O 삽입
            evaluate_label_ids.append(tid) # [0]
            wid += 1
            tid += len(subwords)

        assert tid == len(tokens_a) # subwords의 개수만큼 계속 더했기 때문에, 현재까지 토큰화된 단어의 수와 같을 것
        # 참이면 계속 진행, 아니면 에러 띄움

        evaluate_label_ids = np.array(evaluate_label_ids, dtype=np.int32) # 리스트에서 어레이로 바뀜, [0]
        examples_tokenized.append((tokens_a, labels_a, evaluate_label_ids)) # [(['But'], ['O'], [0]), (['the'], ['O'], [0]), ...]
        if len(tokens_a) > max_seq_length:
            max_seq_length = len(tokens_a) # sequence 길이

    max_seq_length += 2
    for (tokens_a, labels_a, evaluate_label_ids) in examples_tokenized: # (['But'], ['O'], [0])
        tokens = tokens_a + [sep_token] # ['But', '[SEP]']
        segment_ids = [sequence_a_segment_id] * len(tokens) # [0, 0]
        labels = labels_a + ['O'] # sep_token이 더해졌기 때문에 'O' 하나 더 추가, ['O', 'O']

        # if cls_token_at_end, CLS 토큰을 끝에 둘 것임
        tokens = [cls_token] + tokens # ['[CLS]', 'But', '[SEP]']
        segment_ids = [cls_token_segment_id] + segment_ids # [1, 0, 0]
        labels = ['O'] + labels # ['O', 'O', 'O']
        evaluate_label_ids += 1 # [1]

        input_ids = tokenizer.convert_tokens_to_ids(tokens) # 정수 인코딩, [ 101, 2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149, 1012,  102, ...]
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids) # padding 아닌 부분을 1로 채움
        padding_length = max_seq_length - len(input_ids) # padding 할 길이 구함
        label_ids = [label_map[label] for label in labels] # labels를 정수 인코딩함, [0, 0, 9, 0, 0, 0, 0, 0, 0]

        # if pad_on_left, padding을 왼쪽에다 할 것임
        input_ids = input_ids + ([pad_token] * padding_length) # [ 101, 2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149, 1012,  102,    0, ...]
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length) # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length) # [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]

        label_ids = label_ids + ([0] * padding_length) # [0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length # 길이가 다 맞으면 계속함

        features.append(SeqInputFeatures(input_ids=input_ids,
                                         input_mask=input_mask,
                                         segment_ids=segment_ids,
                                         label_ids=label_ids,
                                         evaluate_label_ids=evaluate_label_ids))
    print('maximal sequence length is %d' % (max_seq_length))
    return features

def get_absa_loader(args, tokenizer, mode):
    '''
    Input:
        args: configuration
        tokenizer: tokenizer (Bert-Tokenizer)

    Returns:
        dataset: dataset consist of input_ids, input_mask, segment_ids, label_ids according to dataset
        all_evaluate_label_ids: label_ids (such as E-POS, I-POS, S-NEG, etc..)

    examples:
        (tensor([ 101, 2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149, 1012,  102,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                0,    0,    0,    0,    0]),

         tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),

         tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),

         tensor([0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    '''

    processor = ABSAProcessor()

    # cache' name
    cached_features_file = os.path.join(args.semeval_dir,
                                        'cached_{}_{}_{}'.format(
                                        mode,
                                        list(filter(None, args.model_name_or_path.split('/'))).pop(),
                                        str(args.max_seq_length)))


    if os.path.exists(cached_features_file):
        print('cached_features_file:', cached_features_file)
        features = torch.load(cached_features_file)

    else:
        label_list = processor.get_labels()
        examples = processor.get_examples(args.semeval_dir, mode)

        features = convert_examples_to_seq_features(examples=examples, label_list=label_list, tokenizer=tokenizer,
                                                    cls_token_segment_id=0, pad_token_segment_id=0)

        torch.save(features, cached_features_file) # 파일로 저장 ===========* 여기까지 구현 목표


    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)
    all_evaluate_label_ids = [f.evaluate_label_ids for f in features]
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    if mode=='train':
        sampler = RandomSampler(dataset)
    elif mode=='valid' or mode=='test':
        sampler = SequentialSampler(dataset)

    dataloader = DataLoader(dataset, sampler=sampler, batch_size=args.batch_size)

    return (dataloader, all_evaluate_label_ids)

In [ ]:
class BertABSATagger(BertPreTrainedModel):
    def __init__(self, bert_config):
        """

        :param bert_config: configuration for bert model
        """
        super(BertABSATagger, self).__init__(bert_config)
        self.num_labels = bert_config.num_labels
        self.tagger_config = TaggerConfig()
        self.tagger_config.absa_type = bert_config.absa_type.lower()
        if bert_config.tfm_mode == 'finetune':
            # initialized with pre-trained BERT and perform finetuning
            # print("Fine-tuning the pre-trained BERT...")
            self.bert = BertModel(bert_config)
        else:
            raise Exception("Invalid transformer mode %s!!!" % bert_config.tfm_mode)
        self.bert_dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        # fix the parameters in BERT and regard it as feature extractor
        if bert_config.fix_tfm:
            # fix the parameters of the (pre-trained or randomly initialized) transformers during fine-tuning
            for p in self.bert.parameters():
                p.requires_grad = False

        self.tagger = None
        if self.tagger_config.absa_type == 'linear':
            # hidden size at the penultimate layer
            penultimate_hidden_size = bert_config.hidden_size
        else:
            self.tagger_dropout = nn.Dropout(self.tagger_config.hidden_dropout_prob)
            if self.tagger_config.absa_type == 'lstm':
                self.tagger = LSTM(input_size=bert_config.hidden_size,
                                   hidden_size=self.tagger_config.hidden_size,
                                   bidirectional=self.tagger_config.bidirectional)
            elif self.tagger_config.absa_type == 'gru':
                self.tagger = GRU(input_size=bert_config.hidden_size,
                                  hidden_size=self.tagger_config.hidden_size,
                                  bidirectional=self.tagger_config.bidirectional)
            elif self.tagger_config.absa_type == 'tfm':
                # transformer encoder layer
                self.tagger = nn.TransformerEncoderLayer(d_model=bert_config.hidden_size,
                                                         nhead=12,
                                                         dim_feedforward=4*bert_config.hidden_size,
                                                         dropout=0.1)
            elif self.tagger_config.absa_type == 'san':
                # vanilla self attention networks
                self.tagger = SAN(d_model=bert_config.hidden_size, nhead=12, dropout=0.1)
            elif self.tagger_config.absa_type == 'crf':
                self.tagger = CRF(num_tags=self.num_labels)
            else:
                raise Exception('Unimplemented downstream tagger %s...' % self.tagger_config.absa_type)
            penultimate_hidden_size = self.tagger_config.hidden_size
        self.classifier = nn.Linear(penultimate_hidden_size, bert_config.num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,
                position_ids=None, head_mask=None):
        outputs = self.bert(input_ids, position_ids=position_ids, token_type_ids=token_type_ids,
                            attention_mask=attention_mask, head_mask=head_mask)
        # the hidden states of the last Bert Layer, shape: (bsz, seq_len, hsz)
        tagger_input = outputs[0]
        tagger_input = self.bert_dropout(tagger_input)
        #print("tagger_input.shape:", tagger_input.shape)
        if self.tagger is None or self.tagger_config.absa_type == 'crf':
            # regard classifier as the tagger
            logits = self.classifier(tagger_input)
        else:
            if self.tagger_config.absa_type == 'lstm':
                # customized LSTM
                classifier_input, _ = self.tagger(tagger_input)
            elif self.tagger_config.absa_type == 'gru':
                # customized GRU
                classifier_input, _ = self.tagger(tagger_input)
            elif self.tagger_config.absa_type == 'san' or self.tagger_config.absa_type == 'tfm':
                # vanilla self-attention networks or transformer
                # adapt the input format for the transformer or self attention networks
                tagger_input = tagger_input.transpose(0, 1)
                classifier_input = self.tagger(tagger_input)
                classifier_input = classifier_input.transpose(0, 1)
            else:
                raise Exception("Unimplemented downstream tagger %s..." % self.tagger_config.absa_type)
            classifier_input = self.tagger_dropout(classifier_input)
            logits = self.classifier(classifier_input)
        outputs = (logits,) + outputs[2:]

        if labels is not None:
            if self.tagger_config.absa_type != 'crf':
                loss_fct = CrossEntropyLoss()
                if attention_mask is not None:
                    active_loss = attention_mask.view(-1) == 1
                    active_logits = logits.view(-1, self.num_labels)[active_loss]
                    active_labels = labels.view(-1)[active_loss]
                    loss = loss_fct(active_logits, active_labels)
                else:
                    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                outputs = (loss,) + outputs
            else:
                log_likelihood = self.tagger(inputs=logits, tags=labels, mask=attention_mask)
                loss = -log_likelihood
                outputs = (loss,) + outputs
        return outputs


In [ ]:
# coding=utf-8
# Copyright 2018 Google AI Language, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from transformers import PreTrainedModel, BertModel, BertConfig
from torch.utils.data import Dataset

import torch
from transformers import BertModel
from torch import nn

class BertLayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

class BertPreTrainedModel(PreTrainedModel):
    config_class = BertConfig
    base_model_prefix = "bert"

    def __init__(self, *inputs, **kwargs):
        super(BertPreTrainedModel, self).__init__(*inputs, **kwargs)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

###########################################################################################################################

class TaggerConfig:
    def __init__(self):
        self.hidden_dropout_prob = 0.1
        self.hidden_size = 768

class SAN(nn.Module):
    def __init__(self, model, nhead, dropout=0.1):
        super(SAN, self).__init__()
        self.model = model
        self.self_attn = nn.MultiheadAttention(model, nhead, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)
        self.norm = nn.LayerNorm(model)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask) # (key, query, value)
        src = src + self.dropout(src2)
        src = self.norm(src)
        return src

class BertABSATagger(BertPreTrainedModel):
    def __init__(self, bert_config):
        super(BertABSATagger, self).__init__(bert_config)
        self.num_labels = bert_config.num_labels
        self.tagger_config = TaggerConfig()
        self.bert = BertModel(bert_config)
        self.bert_dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        if bert_config.fix_tfm:
            for p in self.bert.parameters():
                p.required_grad = False  # Frizen

        self.tagger_dropout = nn.Dropout(self.tagger_config.hidden_dropout_prob)
        self.tagger = SAN(model=bert_config.hidden_size, nhead=12, dropout=0.1)
        penultimate_hidden_size = self.tagger_config.hidden_size
        self.classifier = nn.Linear(penultimate_hidden_size, self.num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, position_ids=None, head_mask=None):
        outputs = self.bert(
            input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            head_mask=head_mask
        )

        tagger_input = outputs[0] # pooler
        tagger_input = self.bert_dropout(tagger_input)
        tagger_input = tagger_input.transpose(0, 1) # 각 성분에 대해서 classification을 하기 위함.
        classifier_input = self.tagger(tagger_input)
        classifier_input = classifier_input.transpose(0, 1)
        classifier_input = self.tagger_dropout(classifier_input)
        logits = self.classifier(classifier_input)

        outputs = (logits, ) + outputs[2:]

        if labels is not None:
            criterion = nn.CrossEntropyLoss()

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = criterion(active_logits, active_labels)

            else:
                loss = criterion(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss, ) + outputs

        return outputs


class ABSADataset(Dataset):
    def __init__(self, args, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.reviews = dataframe.text
        self.labels = dataframe.stars
        self.max_seq_length = args.max_seq_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        review = self.reviews[idx]

        inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_seq_length,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        input_ids = inputs['input_ids']
        masks = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return (
            torch.tensor(input_ids, dtype=torch.long), # token_ids
            torch.tensor(masks, dtype=torch.long), # attention_mask
            torch.tensor(token_type_ids, dtype=torch.long), # token_type_ids
            torch.tensor(self.labels[idx], dtype = float) # labels
        )